<a href="https://colab.research.google.com/github/prachi1294/599-Team-1/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import spacy
import nltk
#nltk.download("gutenberg")
#from nltk.corpus import gutenberg
import requests
import re
from collections import defaultdict

In [3]:
nlp = spacy.load('en')
import pandas as pd
import matplotlib.pyplot as plt
import random

In [4]:
def read_books(url):
  r = requests.get(url)
  # What comes back includes headers and other HTTP stuff, get just the body of the response
  t = r.text
  return t


#save all the books
the_extra_ordinary_adv_lupin = 'https://www.gutenberg.org/files/6133/6133-0.txt'
arsene_lupin = 'https://www.gutenberg.org/cache/epub/4014/pg4014.txt'
lupin_vs_herlock_sholmes = 'https://www.gutenberg.org/files/40203/40203-0.txt'
the_hollow_needle = 'https://www.gutenberg.org/files/4017/4017-0.txt'
the_confession_of_lupin = 'https://www.gutenberg.org/cache/epub/28093/pg28093.txt'


#read all the books one by one

book1 = read_books(the_extra_ordinary_adv_lupin)
book2 = read_books(arsene_lupin)
book3 = read_books(lupin_vs_herlock_sholmes)
book4 = read_books(the_hollow_needle)
book5 = read_books(the_confession_of_lupin)




In [12]:
##my regex = (IX|IV|V?I{1,3}\.\s)([A-Z].*)(\W*[a-zA-Z])
pattern = ("(IX|IV|V?I{1,3}\.\s)([A-Z].*)") #get the chapter name. Group 1 will get the chapter no followed by . and space. Group 2 will get chapter name
matches = []
matches = re.findall(pattern, book1)

In [10]:
print(matches)

[('I. ', 'The Arrest of ArsÃ¨ne Lupin\r'), ('II. ', 'ArsÃ¨ne Lupin in Prison\r'), ('III. ', 'The Escape of ArsÃ¨ne Lupin\r'), ('VI. ', 'The Seven of Hearts\r'), ('VII. ', 'Madame Imbertâ\x80\x99s Safe\r'), ('VIII. ', 'The Black Pearl\r'), ('IV', 'E OR\r'), ('IV', 'E NOTICE OF THE POSSIBILITY OF SUCH\r')]


In [21]:
words_pattern = ("(?!\[?[0-9]+[\[\]]?)[A-Za-z]?[a-z][a-z\']*") #extract words from book and ignores numeric and capital letters
words = []
words = re.findall(words_pattern, book1)
print(words[:10])

['The', 'Project', 'Gutenberg', 'e', 'Book', 'of', 'The', 'Extraordinary', 'Adventures', 'of']


In [23]:
sentence_pattern = (("[A-Z][^.]*[\.!?]"))
sentence = []
sentence = re.findall(sentence_pattern, book1)
print(sentence[:10])

['The Project Gutenberg eBook of The Extraordinary Adventures of ArsÃ¨ne Lupin, Gentleman-Burglar, by Maurice Leblanc\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.', 'You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.', 'If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.', 'Title: The Extraordinary Adventures of ArsÃ¨ne Lupin, Gentleman-Burglar\r\n\r\nAuthor: Maurice Leblanc\r\n\r\nRelease Date: November 17, 2002 [eBook #6133]\r\n[Last updated: February 6, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Nathan J.', 'Miller, and David Widger\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE EXTRAORDINARY ADVENTURES OF ARSÃ\x88NE LUPIN ***\r\n\r\

In [30]:
# perform tokenization
from typing import List
def tokenize(book1: str) -> List[List[str]]:
  tokens = []
  sentence_pattern = ('[A-Z][^.]*[\.!?]')
  words_pattern = ('[A-za-z0-9\']+')
  sentences = re.findall(sentence_pattern, book1)
  #print(sentences[:10])

  for i in sentences:
    word_list = re.findall(words_pattern, i)
    tokens.append(word_list)
  return tokens

  
tokenize(book1)

[['The',
  'Project',
  'Gutenberg',
  'eBook',
  'of',
  'The',
  'Extraordinary',
  'Adventures',
  'of',
  'Ars',
  'ne',
  'Lupin',
  'Gentleman',
  'Burglar',
  'by',
  'Maurice',
  'Leblanc',
  'This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'United',
  'States',
  'and',
  'most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions',
  'whatsoever'],
 ['You',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  're',
  'use',
  'it',
  'under',
  'the',
  'terms',
  'of',
  'the',
  'Project',
  'Gutenberg',
  'License',
  'included',
  'with',
  'this',
  'eBook',
  'or',
  'online',
  'at',
  'www'],
 ['If',
  'you',
  'are',
  'not',
  'located',
  'in',
  'the',
  'United',
  'States',
  'you',
  'will',
  'have',
  'to',
  'check',
  'the',
  'laws',
  'of',
  'the',
  'country',
  'where',
  'you',
  'are',
  'located',
  'be